In [22]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [23]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from imblearn.over_sampling import SMOTE 

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

Load dataset

In [24]:
df = pd.read_csv('Telco Customer Churn.csv')
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [26]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [27]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

def feature_to_float(feature_list,df):
    for i in feature_list :
        df[i] = df[i].astype(float)
        return df
    
feature_to_float(['tenure'],df)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1.0,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34.0,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2.0,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45.0,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2.0,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24.0,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72.0,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11.0,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4.0,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [28]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [29]:
df.isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [30]:
df['TotalCharges'].fillna(method='pad', inplace=True)

In [31]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## Data Preprocessing
### Label Encoding

In [34]:
le = preprocessing.LabelEncoder()
le.fit(df['Churn'])
df['Churn_label_encoding'] = le.transform(df['Churn']) 

In [35]:
Churn_Label_Encoding = df[['Churn','Churn_label_encoding',]]
Churn_Label_Encoding.head()

,Churn,Churn_label_encoding
0,No,0
1,No,0
2,Yes,1
3,No,0
4,Yes,1


In [37]:
df['Churn_label_cat'] = df['Churn'].astype('category').cat.codes

In [38]:
Churn_label_cat = df[['Churn','Churn_label_encoding','Churn_label_cat']]
Churn_label_cat.head()

,Churn,Churn_label_encoding,Churn_label_cat
0,No,0,0
1,No,0,0
2,Yes,1,1
3,No,0,0
4,Yes,1,1


### One Hot Encoding

In [41]:
dummies_Churn = pd.get_dummies(df['Churn'],prefix='Churn')
dummies_Churn.head()

,Churn_No,Churn_Yes
0,1,0
1,1,0
2,0,1
3,1,0
4,0,1


In [43]:
df = pd.concat([df, dummies_Churn], axis=1)
df[['Churn','Churn_No','Churn_Yes']].head()

,Churn,Churn_No,Churn_No,Churn_Yes,Churn_Yes
0,No,1,1,0,0
1,No,1,1,0,0
2,Yes,0,0,1,1
3,No,1,1,0,0
4,Yes,0,0,1,1


### Frequency Encoding

In [47]:
freq_et = df['Churn'].value_counts().reset_index()
freq_et.rename(columns={"index": "Churn", "Churn": "freq_Churn"}, inplace = True)
freq_et['pct_Churn'] = round((freq_et['freq_Churn']/freq_et['freq_Churn'].sum())*100,2)
freq_et

,Churn,freq_Churn,pct_Churn
0,No,5174,73.46
1,Yes,1869,26.54


In [48]:
df = df.merge(freq_et[['Churn','pct_Churn']], on='Churn', how='inner')
df[['Churn','pct_Churn']]

,Churn,pct_Churn
0,No,73.46
1,No,73.46
2,No,73.46
3,No,73.46
4,No,73.46
...,...,...
7038,Yes,26.54
7039,Yes,26.54
7040,Yes,26.54
7041,Yes,26.54


In [49]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Churn,Churn_label_cat,Churn_label_encoding,Churn_No,Churn_Yes,Churn_No,Churn_Yes,pct_Churn_x,pct_Churn_y,pct_Churn
0,7590-VHVEG,Female,0,Yes,No,1.0,No,No phone service,DSL,No,...,No,0,0,1,0,1,0,73.46,73.46,73.46
1,5575-GNVDE,Male,0,No,No,34.0,Yes,No,DSL,Yes,...,No,0,0,1,0,1,0,73.46,73.46,73.46
2,7795-CFOCW,Male,0,No,No,45.0,No,No phone service,DSL,Yes,...,No,0,0,1,0,1,0,73.46,73.46,73.46
3,1452-KIOVK,Male,0,No,Yes,22.0,Yes,Yes,Fiber optic,No,...,No,0,0,1,0,1,0,73.46,73.46,73.46
4,6713-OKOMC,Female,0,No,No,10.0,No,No phone service,DSL,Yes,...,No,0,0,1,0,1,0,73.46,73.46,73.46


## Imbalanced Data handling

In [51]:
target = df['Churn_label_encoding'].value_counts().reset_index()
target.rename(columns={'index':'Churn_label_encoding','Churn_label_encoding':'freq'}, inplace=True)
target['percentage'] = round((target['freq']/target['freq'].sum())*100,2)
target

,Churn_label_encoding,freq,percentage
0,0,5174,73.46
1,1,1869,26.54


In [55]:
df_numeric = df[['Churn_label_encoding','tenure','TotalCharges','MonthlyCharges']]
df_numeric = df_numeric.dropna()
df_numeric.head(10)

,Churn_label_encoding,tenure,TotalCharges,MonthlyCharges
0,0,1.0,29.85,29.85
1,0,34.0,1889.50,56.95
2,0,45.0,1840.75,42.30
3,0,22.0,1949.40,89.10
4,0,10.0,301.90,29.75
5,0,62.0,3487.95,56.15
6,0,13.0,587.45,49.95
7,0,16.0,326.80,18.95
8,0,58.0,5681.10,100.35
9,0,25.0,2686.05,105.50


In [57]:
df_numeric.tail()

,Churn_label_encoding,tenure,TotalCharges,MonthlyCharges
7038,1,12.0,727.80,59.80
7039,1,9.0,403.35,44.20
7040,1,1.0,75.75,75.75
7041,1,67.0,6886.25,102.95
7042,1,4.0,306.60,74.40


In [59]:
df_numeric['Churn_label_encoding'].value_counts()

0    5174
1    1869
Name: Churn_label_encoding, dtype: int64

In [60]:
from imblearn import under_sampling, over_sampling

X = df_numeric.drop(['Churn_label_encoding'],axis=1)
y = df_numeric['Churn_label_encoding']

# Undersampling
X_under, y_under = under_sampling.RandomUnderSampler().fit_resample(X, y)

# Overampling
X_over, y_over = over_sampling.RandomOverSampler().fit_resample(X, y)

# Overampling with smote
X_over_smote, y_over_smote = over_sampling.SMOTE().fit_resample(X, y)

In [61]:
df_undersampling = pd.concat([X_under, y_under], axis=1)
df_undersampling.head(10)

,tenure,TotalCharges,MonthlyCharges,Churn_label_encoding
0,49.0,3902.45,79.30,0
1,12.0,444.75,33.15,0
2,65.0,6416.70,99.05,0
3,1.0,20.50,20.50,0
4,17.0,475.25,24.80,0
5,3.0,76.95,20.50,0
6,30.0,3096.90,100.45,0
7,24.0,482.80,20.40,0
8,68.0,7849.85,114.70,0
9,12.0,893.00,76.60,0


In [62]:
df_oversampling = pd.concat([X_over, y_over], axis=1)
df_oversampling.head(10)

,tenure,TotalCharges,MonthlyCharges,Churn_label_encoding
0,1.0,29.85,29.85,0
1,34.0,1889.50,56.95,0
2,45.0,1840.75,42.30,0
3,22.0,1949.40,89.10,0
4,10.0,301.90,29.75,0
5,62.0,3487.95,56.15,0
6,13.0,587.45,49.95,0
7,16.0,326.80,18.95,0
8,58.0,5681.10,100.35,0
9,25.0,2686.05,105.50,0


In [63]:
df_oversampling_smote = pd.concat([X_over_smote, y_over_smote], axis=1)
df_oversampling_smote.head(10)

,tenure,TotalCharges,MonthlyCharges,Churn_label_encoding
0,1.0,29.85,29.85,0
1,34.0,1889.50,56.95,0
2,45.0,1840.75,42.30,0
3,22.0,1949.40,89.10,0
4,10.0,301.90,29.75,0
5,62.0,3487.95,56.15,0
6,13.0,587.45,49.95,0
7,16.0,326.80,18.95,0
8,58.0,5681.10,100.35,0
9,25.0,2686.05,105.50,0


In [65]:
df_undersampling['Churn_label_encoding'].value_counts()

0    1869
1    1869
Name: Churn_label_encoding, dtype: int64

In [67]:
df_oversampling['Churn_label_encoding'].value_counts()

0    5174
1    5174
Name: Churn_label_encoding, dtype: int64

In [69]:
df_oversampling_smote['Churn_label_encoding'].value_counts()

0    5174
1    5174
Name: Churn_label_encoding, dtype: int64

### Modeling (Gunakan lebih min 2 model dan bandingkan hasil evaluasinya)
Bebas menggunakan model, mau menggunakan decision tree, random forest, xgboost, dll juga boleh

silahkan berekspresi :)

### Random Forest with balance data

In [71]:
df = pd.read_csv('Telco Customer Churn.csv')
df = df.dropna().reset_index(drop=True)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [72]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

### Data preproc - Gender

In [87]:
df = pd.read_csv('Telco Customer Churn.csv')
df = df.dropna().reset_index(drop=True)
df['gender'] = df['gender'].replace("Female", 0).replace("Male", 1)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,1,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,1,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,1,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,0,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [88]:
df =pd.get_dummies(data=df,columns=['customerID', 'Partner', 'Dependents',
                                      'PhoneService','MultipleLines','InternetService',
                                      'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'],drop_first=True)
df.head()

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,customerID_0003-MKNFE,customerID_0004-TLHLJ,customerID_0011-IGKFF,customerID_0013-EXCHZ,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,29.85,29.85,No,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,34,56.95,1889.5,No,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,2,53.85,108.15,Yes,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,1,0,45,42.30,1840.75,No,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,2,70.70,151.65,Yes,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [90]:
le = preprocessing.LabelEncoder()
le.fit(df['Churn'])
df['Churn_label_encoding'] = le.transform(df['Churn']) 

In [91]:
Churn_Label_Encoding = df[['Churn','Churn_label_encoding',]]
Churn_Label_Encoding.head()

,Churn,Churn_label_encoding
0,No,0
1,No,0
2,Yes,1
3,No,0
4,Yes,1


In [92]:
df.head()

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,customerID_0003-MKNFE,customerID_0004-TLHLJ,customerID_0011-IGKFF,customerID_0013-EXCHZ,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_label_encoding
0,0,0,1,29.85,29.85,No,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,1,0,34,56.95,1889.5,No,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0,2,53.85,108.15,Yes,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
3,1,0,45,42.30,1840.75,No,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,2,70.70,151.65,Yes,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [94]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

def feature_to_float(feature_list,df):
    for i in feature_list :
        df[i] = df[i].astype(float)
        return df
    
feature_to_float(['TotalCharges'],df)

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,customerID_0003-MKNFE,customerID_0004-TLHLJ,customerID_0011-IGKFF,customerID_0013-EXCHZ,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_label_encoding
0,0,0,1,29.85,29.85,No,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,1,0,34,56.95,1889.50,No,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0,2,53.85,108.15,Yes,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
3,1,0,45,42.30,1840.75,No,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,2,70.70,151.65,Yes,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,24,84.80,1990.50,No,0,0,0,0,...,1,0,1,1,0,1,0,0,1,0
7039,0,0,72,103.20,7362.90,No,0,0,0,0,...,1,0,1,1,0,1,1,0,0,0
7040,0,0,11,29.60,346.45,No,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
7041,1,1,4,74.40,306.60,Yes,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1


In [98]:
del(df['Churn'])

In [101]:
df.head()

,gender,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,customerID_0003-MKNFE,customerID_0004-TLHLJ,customerID_0011-IGKFF,customerID_0013-EXCHZ,customerID_0013-MHZWF,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_label_encoding
0,0,0,1,29.85,29.85,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,1,0,34,56.95,1889.50,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,0,2,53.85,108.15,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
3,1,0,45,42.30,1840.75,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,2,70.70,151.65,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [102]:
df['TotalCharges'].fillna(method='pad', inplace=True)

### Balanced Data

In [104]:
sm = over_sampling.SMOTE(random_state=42)

X = df.drop(['Churn_label_encoding'],axis = 1)
Y = df['Churn_label_encoding']
X_sm, y_sm = sm.fit_resample(X, Y)

print(f'''Shape of X before SMOTE: {X.shape}
Shape of X after SMOTE: {X_sm.shape}''')

print('\nBalance of positive and negative classes (%):')
y_sm.value_counts(normalize=True) * 100


Shape of X before SMOTE: (7043, 7072)
Shape of X after SMOTE: (10348, 7072)

Balance of positive and negative classes (%):


0    50.0
1    50.0
Name: Churn_label_encoding, dtype: float64

In [105]:
y_sm.value_counts()

0    5174
1    5174
Name: Churn_label_encoding, dtype: int64

## Split Dataset

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

## Modeling dengan Random Forest

In [107]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

### Metric Evaluation
Karena sudah balanced data, jadi valid menggunakan akurasi

In [108]:
print('Akurasi',accuracy_score(y_test, preds))

Akurasi 0.8422883649014302


### Random Forest with Imbalance data

In [142]:
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, recall_score, precision_score

In [133]:
df['Churn_label_encoding'].value_counts()

0    5174
1    1869
Name: Churn_label_encoding, dtype: int64

### Split Dataset

In [134]:
X = df.drop(['Churn_label_encoding'],axis = 1)
Y = df['Churn_label_encoding']

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

### Modeling dengan Random Forest

In [136]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

### Metric Evaluation
Karena sudah tidak balanced data, jadi TIDAK valid menggunakan akurasi


gunakan AUC atau F1 score

In [137]:
cm = confusion_matrix(y_test, preds)
cm

array([[1184,   98],
       [ 268,  211]], dtype=int64)

In [138]:
# calculate AUC
print('AUC',roc_auc_score(y_test, preds))


AUC 0.6820289930595136


In [139]:
# calculate F1 Score
print('F1 Score ',f1_score(y_test, preds))


F1 Score  0.5355329949238578


In [143]:
# calculate Precision
print('Precision ',precision_score(y_test, preds))

Precision  0.6828478964401294


### jika dievaluasi, maka lebih bagus menggunakan balanced data dengan matrik evaluasi akurasi dg hasil 84%

### Modeling dengan Decision Tree

In [155]:
from sklearn.tree import DecisionTreeClassifier

df = DecisionTreeClassifier(random_state=42)
df.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [157]:
y_pred_df = df.predict(X_test)
y_pred_df

array([1, 0, 0, ..., 0, 1, 1])

In [158]:
print('Akurasi Decission Tree',accuracy_score(y_test, y_pred_df))

Akurasi Decission Tree 0.7734241908006815


In [159]:
#Calculate AUC
print('AUC',roc_auc_score(y_test, preds))

AUC 0.6820289930595136


In [162]:
#Calculate F1 Score
print('F1 Score',f1_score(y_test, preds))

F1 Score 0.5355329949238578


In [163]:
#calculate Precision
print('Precision ',precision_score(y_test, preds))

Precision  0.6828478964401294


In [165]:
#Calculate Recall
print('Recall',recall_score(y_test, preds))

Recall 0.4405010438413361


In [ ]:
#jika dievaluasi, maka lebih bagus menggunakan matrik evaluasi akurasi dg hasil 77%